# Webapp for Dog Breed Identification

Upload a dog photo from your computer and see if we predict the dog breed correctly!

In [2]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [3]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1KgRs1fpWCqkjGNHhJfQb_QsA5kyVNnF4'
destination = 'model.pkl'
download_file_from_google_drive(file_id, destination)

In [4]:
model_path = Path('./model.pkl')
learn_inf = load_learner(model_path, cpu=True)

In [5]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [6]:
def on_data_change(change):
    lbl_pred.value = ''
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    # lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    lbl_pred.value = f'Prediction: {pred}'

In [7]:
btn_upload.observe(on_data_change, names=['data'])

In [8]:
display(VBox([widgets.Label('Select your dog!'), lbl_pred, btn_upload, out_pl]))